<a href="https://colab.research.google.com/github/Arslane101/PFE/blob/main/ENCCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import calendar
import json
import math
import os
from os import cpu_count
import pickle as pk
from math import nan
from re import I
from unittest import result
import random
from keras.engine.input_layer import Input
from datetime import date,datetime,timedelta
import pandas as pd
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from yaml import load
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!rm *

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

In [43]:
def ChargerDataset(ratings,th):
    for i in range(ratings.shape[0]):
        if pd.isnull(ratings['rating'][i]) :
            ratings.loc[i,'rating'] = int(0)
        if int(ratings['rating'][i]) >= int(th):
            ratings.loc[i,'rating']= int(1)
        else: ratings.loc[i,'rating']=int(0) 
def ListRelevant(matrix,n_items,ind):
    relevants = []
    for i in range(n_items):
        if(matrix.iloc[ind,i]==1):
            relevants.append(i)
    return relevants   
def ListRel(array):
    relevants = []
    for i in range(len(array)):
        if(array[i]==1):
            relevants.append(i)
    return relevants 
def ListSpecRel(array):
    relevants = []
    for i in range(len(array)):
        if(array[i]==1):
            relevants.append(list_movies[i])
    return relevants 
def Relevant(matrix):
    relevants = []
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if(matrix.iloc[i,j]==1) and j not in relevants:
              relevants.append(j)
    return relevants   
def Commons(subset,subsets):
    count = 0
    for i in range(len(subsets)):
        if(len(set(subset).intersection(subsets[i]))!=0):
            count+=1
    return count
def RandomSubsets(n_items,nb):
    subsets = list()
    subset = list(range(0,n_items))
    for i in range(nb):
        sub = random.sample(subset,int(n_items/nb))
        subset = list(set(subset)-set(sub))
        subsets.append(sub)
    return subsets
def where(arr,nb):
    for i in range(len(arr)):
        if(arr[i]==nb):
            return i
def EnsembleSamplesTraining(nb):
  itemslist = np.loadtxt("Subsets.txt")
  nbrel= ratings[ratings["rating"] == 1.0].shape[0]
  k=0
  Input = np.zeros((nbrel,n_items),dtype=np.int8)
  Target = np.zeros((nbrel),dtype=np.int16)
  for i in range(pivot.shape[0]):
    for j in  ListRelevant(pivot,n_items,i):
        Input[k] = np.array(pivot.iloc[i,:],copy=True)
        Input[k,j]=0
        Target[k]=j
        k+=1
  print(Input.shape) 
  print(Target.shape)     
  #Splitting the Data
  i=0
  for  i in range(nb):
   if(nb != 1 ) :
     itembis = itemslist[i]
   else : itembis = itemslist
   count = 0
   for j in range(len(Target)):
        if(Target[j] in itembis):
                count+=1
   InputT = np.zeros((count,n_items))
   TargetT= np.zeros((count))
   k=0
   j=0       
   while j<len(InputT) or k<count:
        if(Target[j] in itembis):
            InputT[k]=Input[j,:]
            TargetT[k]=where(itembis,Target[j])
            k+=1
        j+=1
   InputTrain,InputTest,TargetTrain,TargetTest = train_test_split(InputT,TargetT,test_size=0.2,random_state=28)
   np.savetxt("InputTe"+str(i)+".txt",InputTest.astype(int),fmt='%d')
   np.savetxt("TargetTe"+str(i)+".txt",TargetTest.astype(int),fmt='%d')
   np.savetxt("InputTr"+str(i)+".txt",InputTrain.astype(int),fmt='%d')
   np.savetxt("TargetTr"+str(i)+".txt",TargetTrain.astype(int),fmt='%d')
def EnsembleSamplesTesting(nb):
    itemslist = np.loadtxt("Subsets.txt")
    values = list()
    itemlist = np.concatenate(itemslist)
    shape = itemslist.shape[0]
    for i in range(shape):
        model = load_model(str(i))
        testUser = np.array(pivot.iloc[nb,:],copy=True)
        testUser = testUser.reshape(1,testUser.shape[0])
        results = model.predict(testUser)
        results = results/itemslist.shape[1]
        values.append(results)
    results = np.concatenate(np.asarray(values))
    results = np.argsort(results.reshape(itemlist.shape[0]))[::-1] 
    for i in range(results.shape[0]):
        results[i] = int(itemlist[results[i]]) 
    return results
def EnsembleLearning(nb):
 itembis = np.loadtxt("Subsets.txt")
 i=0
 for i in range(nb):
  if nb != 1 :
    liste = itembis[i]
  else : liste = itembis
  InputTest = np.loadtxt("InputTe"+str(i)+".txt")
  TargetTest = np.loadtxt("TargetTe"+str(i)+".txt")
  InputTrain =  np.loadtxt("InputTr"+str(i)+".txt")
  TargetTrain = np.loadtxt("TargetTr"+str(i)+".txt")
  model = Sequential()
  model.add(Input(shape=InputTrain.shape[1]))
  model.add(Dense(100, activation='relu'))
  model.add(Dropout(rate=0.2))
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(rate=0.2))
  model.add(Dense(len(liste),activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
  model.summary()
  model.fit(InputTrain,TargetTrain,validation_data=(InputTest,TargetTest),epochs=80,batch_size=150)
  model.save(format(i))
def MostRatedMovies(ratings,min):
    ratings = ratings.groupby(['movieId'])[['rating']].mean()
    ratings = ratings[ratings["rating"] >= min]
    return ratings.index.unique().tolist()
def FilterContext(results,movies):
    movie = list()
    for mov in movies:
        movie.append(where(list_movies,mov))
    result = list()
    for elt in results:
        if( elt in movie):
            result.append(elt)
    return result
def EnsembleSamples(nb):
    itemslist = np.loadtxt("Subsets.txt")
    itemlist = np.concatenate(itemslist)
    values = list()
    for i in range(itemslist.shape[0]):
        model = load_model(str(i))
        testUser = np.array(pivot.iloc[nb,:],copy=True)
        testUser = testUser.reshape(1,testUser.shape[0])
        results = model.predict(testUser)
        values.append(results)
    results = np.concatenate(np.asarray(values))
    results = results.reshape(itemlist.shape[0])
    result = pd.DataFrame(columns=['movieId','probability'])
    for i in range(results.shape[0]):
        result.loc[len(result.index)]=[list_movies[int(itemlist[i])],results[i]]
    return result

In [ ]:
ratings = pd.read_csv("ratings.csv",delimiter=";",parse_dates=['timestamp'],infer_datetime_format=True)
print(ratings.shape)


In [5]:
ChargerDataset(ratings,4)

In [22]:
pivot = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating',fill_value=0)
n_users = pivot.index.unique().shape[0]
n_items = pivot.columns.unique().shape[0]
list_movies = pivot.columns.unique().tolist()
list_users = pivot.index.unique().tolist()

subsets = RandomSubsets(n_items,2)
np.savetxt("Subsets.txt",np.array(subsets).astype(int),fmt='%d')


In [7]:
print(pivot.shape)


(943, 1682)


In [44]:
EnsembleSamplesTraining(2)

(55375, 1682)
(55375,)


In [ ]:
EnsembleLearning(2)

In [ ]:
j=0
n= 96
totalprec = list()
totalrec = list()
for j in range(40):
 i=1
 recalls = list()
 precisions = list()
 recalls.append(j)
 precisions.append(j)
 testUser = np.array(pivot.iloc[j,:],copy=True)
 rev  = ListRel(testUser)
 results = EnsembleSamplesTesting(j)
 if(len(rev)>50):
  recalls.append(len(rev))
  precisions.append(len(rev))    
  while(i<n):   
    hr=0
    temp = results[:i] 
    for k in range(len(temp)):
         if  temp[k] in rev:
          hr+=1
    prec = (hr)/i
    rec =  (hr)/len(rev) 
    precisions.append(prec)
    recalls.append(rec)
    i+=5
  totalprec.append(np.asarray(precisions))
  totalrec.append(np.asarray(recalls))
np.savetxt("AllPrecisions.txt", np.vstack(totalprec).astype(float),fmt='%.2f')
np.savetxt("AllRecalls.txt",np.vstack(totalrec).astype(float),fmt='%.2f')

1/1 [==============================] - 0s 71ms/step
